In [1]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver import test_driver
from pyvisa import ResourceManager, VisaIOError

Could not load Keysight SD1
Could not load Keysight SD1
pylablib not found, AttocubeANC350 not loaded
Basler Camera software not found, BaserCamera not loaded
Helios Camera not installed
msl not installed, Thorlabs BSC203 driver not loaded
seabreeze module not found, Ocean Optics not imported
Failed to load spinapi library.
spinapi is not installed, PulseBlaster driver not loaded.
Thorlabs Kinesis not found, ThorlabsBSC203 not loaded
Thorlabs Kinesis not found, ThorlabsBPC303 not loaded
Thorlabs Kinesis not found, ThorlabsMFF101 not loaded


In [2]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

VisaIOError: VI_ERROR_RSRC_BUSY (-1073807246): The resource is valid, but VISA cannot currently access it.

In [ ]:
srs830 = ps.Stanford830(res)

test_driver(srs830)

Initial state for the Stanford830 was: [('phase', 180.0), ('reference_source', 'external'), ('frequency', 102000.0), ('reference_slope', 'sine zero'), ('harmonic', 1), ('amplitude', 4.996), ('input_configuration', 'A-B'), ('input_ground', 'AC'), ('input_coupling', 'AC'), ('input_line_filter', 'none'), ('sensitivity', 2e-09), ('reserve_mode', 'high'), ('time_constant', 30), ('filter_slope', 6), ('synchronous_filter', 'off'), ('display1_output_source', 'x'), ('display2_output_source', 'y'), ('auxillary_voltage1', -10.5), ('auxillary_voltage2', -10.5), ('auxillary_voltage3', -10.5), ('auxillary_voltage4', -10.5), ('sample_rate', 0.0625), ('end_buffer_mode', 'one shot'), ('trigger_mode', 'off'), ('local_remote_control', 'remote'), ('gpib_overrided_state', 1), ('power_on_status_clear', 'on')]
These blacklisted settings and their corresponding values were not reset:  [('_amplitude', 4.996), ('_input_configuration', 'A-B'), ('_time_constant', 30)]
Reset state for the Stanford830 was: [('phase

AttributeError: 'dict' object has no attribute 'index'